In [1]:
import torch
import torch.nn as nn
import torch.utils.data
import torchvision
from functools import partial
from torch import optim
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms

tensorboard --logdir=runs

In [2]:
writer = SummaryWriter()
writer.add_scalar('example', 3)

In [4]:
import random
value = 10
writer.add_scalar('test_loop', value, 0)
for i in range(1, 10000):
    value += random.random() - 0.5
    writer.add_scalar('test_loop', value, i)

In [5]:
# resnet visualisation - необходимо передать саму модель и тензор для трассировки графа
from torchvision import models

model = models.resnet18(False)
writer.add_graph(model, torch.rand([1, 3, 224, 224]))

#### Использование хуков для получения информации о модели

In [8]:
def print_hook(self, module, inp, outp):
    print('Shape input is {0}'.format(inp.shape))

# применяем хук при прямом проходе, сохраняем ссылку на объект хука
model(torch.rand([1, 3, 224, 224]))
hook_ref = model.fc.register_forward_hook(print_hook)
hook_ref.remove() # убираем хук
model(torch.rand([1, 3, 224, 224]))

tensor([[-4.0559e-01, -5.4750e-01,  1.7231e-01, -1.1816e-01,  9.6418e-01,
         -7.6533e-01,  4.5932e-02,  3.3473e-01,  3.0338e-02,  1.9006e-01,
         -7.4424e-01, -6.2147e-01,  1.0637e-01, -4.4457e-01, -2.2575e-01,
         -8.9592e-02, -4.3492e-01, -3.7914e-02, -5.7275e-01, -8.6920e-01,
          2.2201e-01,  5.2262e-01, -2.1334e-01,  3.5553e-01, -3.9271e-02,
         -1.5388e-02, -6.8094e-01,  2.8797e-01, -3.5601e-01, -3.2038e-01,
         -4.2639e-02, -1.5817e-01, -9.7811e-01, -8.0342e-01, -1.9465e-01,
          6.0866e-01, -2.8846e-01, -3.9827e-02, -3.9155e-01, -1.8197e-02,
         -7.6056e-01, -3.4742e-01,  3.2052e-01, -3.5247e-01, -8.7206e-01,
          1.0139e+00, -6.0379e-01, -9.9967e-01, -3.9774e-02, -3.8887e-01,
          2.5508e-01, -3.8452e-01, -3.2984e-02, -5.8198e-01, -3.2430e-01,
         -8.3038e-02, -1.7649e-01,  6.8163e-01,  7.3631e-01, -4.8466e-01,
         -3.6978e-01, -8.6214e-01,  4.6410e-01,  3.9925e-01,  2.8768e-01,
          1.9155e-01, -3.9436e-02, -1.

#### Хуки для tensorboard для получения статистики

In [9]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))]
)
trainset = datasets.MNIST("mnist_train", train=True, download=True, transform=transform)
train_data_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
model = torchvision.models.resnet50(False)

Extracting mnist_train/MNIST/raw/train-images-idx3-ubyte.gz to mnist_train/MNIST/raw


Extracting mnist_train/MNIST/raw/train-labels-idx1-ubyte.gz to mnist_train/MNIST/raw


Extracting mnist_train/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist_train/MNIST/raw


Extracting mnist_train/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist_train/MNIST/raw
Processing...
Done!





/opt/conda/conda-bld/pytorch_1591914855613/work/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [10]:
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
images, labels = next(iter(train_data_loader))

In [11]:
grid = torchvision.utils.make_grid(images)
writer.add_image("images", grid, 0)
writer.add_graph(model, images)

In [12]:
def send_stats(i, module, input, output):
    writer.add_scalar(f"layer {i}-mean", output.data.mean())
    writer.add_scalar(f"layer {i}-stddev", output.data.std())


for i, m in enumerate(model.children()):
    m.register_forward_hook(partial(send_stats, i))

In [13]:
# Now train the model and watch output in Tensorboard

optimizer = optim.Adam(model.parameters(), lr=2e-2)
criterion = nn.CrossEntropyLoss()


def train(
    model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cuda:0"
):
    model.to(device)
    for epoch in range(epochs):
        print(f"epoch {epoch+1}")
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            ww, target = batch
            ww = ww.to(device)
            target = target.to(device)
            output = model(ww)
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()

        model.eval()
        num_correct = 0
        num_examples = 0
        for batch in val_loader:
            ww, target = batch
            ww = ww.to(device)
            target = target.to(device)
            output = model(ww)
            correct = torch.eq(torch.max(output, dim=1)[1], target).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        print("Epoch {}, accuracy = {:.2f}".format(epoch+1, num_correct / num_examples))


train(model, optimizer, criterion, train_data_loader, train_data_loader, epochs=5)

epoch 1
Epoch 1, accuracy = 0.88
epoch 2
Epoch 2, accuracy = 0.98
epoch 3
Epoch 3, accuracy = 0.95
epoch 4
Epoch 4, accuracy = 0.97
epoch 5
Epoch 5, accuracy = 0.98
